In [1]:
// Now create and display your interactive widget
display("""
    <div style="font-family: sans-serif; padding: 20px; border: 2px solid #4285f4; border-radius: 10px; background: #f8f9fa;">
      <h3>Java Interactive Widget</h3>
      <p>Move the slider (client-side computation):</p>
      <input type="range" min="-50" max="50" value="0" id="slider" style="width: 300px;">
      <span style="font-size: 1.5em; margin-left: 20px;">Value: <span id="val">0</span></span>
      
      <p>Square: <span id="square">0</span></p>
      <p>Cube: <span id="cube">0</span></p>
      
      <script>
        const slider = document.getElementById('slider');
        const val = document.getElementById('val');
        const square = document.getElementById('square');
        const cube = document.getElementById('cube');
        
        function update() {
          const v = parseInt(slider.value);
          val.textContent = v;
          square.textContent = v * v;
          cube.textContent = v * v * v;
        }
        
        slider.oninput = update;
        update();
      </script>
    </div>
    """, "text/html");

<div style="font-family: sans-serif; padding: 20px; border: 2px solid #4285f4; border-radius: 10px; background: #f8f9fa;">
  <h3>Java Interactive Widget</h3>
  <p>Move the slider (client-side computation):</p>
  <input type="range" min="-50" max="50" value="0" id="slider" style="width: 300px;">
  <span style="font-size: 1.5em; margin-left: 20px;">Value: <span id="val">0</span></span>

  <p>Square: <span id="square">0</span></p>
  <p>Cube: <span id="cube">0</span></p>

  <script>
    const slider = document.getElementById('slider');
    const val = document.getElementById('val');
    const square = document.getElementById('square');
    const cube = document.getElementById('cube');

    function update() {
      const v = parseInt(slider.value);
      val.textContent = v;
      square.textContent = v * v;
      cube.textContent = v * v * v;
    }

    slider.oninput = update;
    update();
  </script>
</div>

2026d41d-1745-4f6f-8a36-aa47ec6eff56

In [2]:
String id = display("<div id='prog'>Progress: 0%</div>", "text/html");

for (int i = 0; i <= 100; i += 10) {
    updateDisplay(id, "<div id='prog'>Progress: " + i + "%<progress value='" + i + "' max='100'></progress></div>", "text/html");
    Thread.sleep(500);
}

<div id='prog'>Progress: 100%<progress value='100' max='100'></progress></div>

## Interactive Java Widgets in JupyterLab (JJava)

This notebook demonstrates how to build **interactive UI controls** (e.g., a slider) in **JupyterLab** that communicate with a **Java (JJava) kernel**.

**How it works**
1. HTML + JavaScript are rendered in a cell output using `render(..., "text/html")` and `render(..., "application/javascript")`.
2. The JavaScript accesses the running JupyterLab app via `window.jupyterapp` (enabled with `jupyter lab --expose-app-in-browser`).
3. UI events (slider changes) invoke Java code using `kernel.requestExecute(...)`.
4. The Java method `onChange(int v)` receives the value, performs computation or stores state, and updates existing outputs using `updateDisplay(...)`.

**Key points**
- `render(...)` is used for UI so the cell shows rich output (not a display id).
- `display(...)` + `updateDisplay(...)` are used when an output must be updated in place.
- The notebook must be **trusted** for JavaScript execution.
- Slider values can be stored in Java (e.g., `AtomicInteger`) and reused by other methods or actions.

This pattern enables lightweight, notebook-native interactivity without writing a custom JupyterLab extension.



In [3]:
public class SliderDemo {

    static String displayId;

    public static void init() {
        displayId = display("<div style='font-family:sans-serif'>Value: <b>50</b></div>", "text/html");
    }

    public static void onChange(int v) {
        var multiV = v * 10;
        updateDisplay(displayId,
            "<div style='font-family:sans-serif'>Value: <b>" + multiV + "</b></div>",
            "text/html"
        );
    }
}

SliderDemo.init();


<div style='font-family:sans-serif'>Value: <b>210</b></div>

In [4]:
String anchorId = "jjava-slider-anchor";
render("<div id=\"" + anchorId + "\"></div>", "text/html");


<div id="jjava-slider-anchor"></div>

In [5]:
String anchorId = "jjava-slider-anchor";

String js =
"(async () => {\n" +
"  const host = document.getElementById('" + anchorId + "');\n" +
"  if (!host) { console.error('Anchor not found'); return; }\n" +
"\n" +
"  if (!window.jupyterapp) {\n" +
"    console.error('window.jupyterapp is undefined. Start JupyterLab with --expose-app-in-browser');\n" +
"    return;\n" +
"  }\n" +
"\n" +
"  const root = document.createElement('div');\n" +
"  root.style.cssText = 'margin:8px 0;padding:8px;border:1px solid #ddd;border-radius:6px;font-family:sans-serif;';\n" +
"  root.innerHTML = `\n" +
"    <div style=\"margin-bottom:6px;\"><b>Slider → Java (JJava, JupyterLab)</b></div>\n" +
"    <input id=\"sld\" type=\"range\" min=\"0\" max=\"100\" value=\"50\" style=\"width:320px\" />\n" +
"    <span id=\"lbl\" style=\"margin-left:10px;\">50</span>\n" +
"  `;\n" +
"  host.appendChild(root);\n" +
"\n" +
"  const slider = root.querySelector('#sld');\n" +
"  const label  = root.querySelector('#lbl');\n" +
"\n" +
"  const session = window.jupyterapp.shell.currentWidget.context.sessionContext.session;\n" +
"  const kernel = window.jupyterapp.serviceManager.sessions._connectToKernel({\n" +
"    model: { id: session._kernel._id, name: session._kernel._name }\n" +
"  });\n" +
"\n" +
"  let t = null;\n" +
"  slider.addEventListener('input', () => {\n" +
"    const v = Number(slider.value);\n" +
"    label.textContent = String(v);\n" +
"    if (t) clearTimeout(t);\n" +
"    t = setTimeout(() => {\n" +
"      kernel.requestExecute({ code: `SliderDemo.onChange(${v});` });\n" +
"    }, 40);\n" +
"  });\n" +
"})();";

render(js, "application/javascript");


(async () => {
  const host = document.getElementById('jjava-slider-anchor');
  if (!host) { console.error('Anchor not found'); return; }

  if (!window.jupyterapp) {
    console.error('window.jupyterapp is undefined. Start JupyterLab with --expose-app-in-browser');
    return;
  }

  const root = document.createElement('div');
  root.style.cssText = 'margin:8px 0;padding:8px;border:1px solid #ddd;border-radius:6px;font-family:sans-serif;';
  root.innerHTML = `
    <div style="margin-bottom:6px;"><b>Slider → Java (JJava, JupyterLab)</b></div>
    <input id="sld" type="range" min="0" max="100" value="50" style="width:320px" />
    <span id="lbl" style="margin-left:10px;">50</span>
  `;
  host.appendChild(root);

  const slider = root.querySelector('#sld');
  const label  = root.querySelector('#lbl');

  const session = window.jupyterapp.shell.currentWidget.context.sessionContext.session;
  const kernel = window.jupyterapp.serviceManager.sessions._connectToKernel({
    model: { id: sessi